# 🚀 HVAC AI — Production-Ready YOLOv11-OBB Inference Pipeline
**End-to-End Oriented Bounding Box (OBB) Analysis for HVAC Blueprints**

---

## 📋 Overview
This notebook provides a complete, production-grade pipeline for analyzing HVAC diagrams using **YOLOv11-OBB**. Unlike standard object detection, OBB handles rotated components (VAVs, diffusers, thermostats) precisely, ensuring tight bounding box alignment.

### 🌟 Key Enhancements
- **True OBB Support**: Handles `xyxyxyxy` polygon coordinates for rotated detection.
- **FP16 Optimization**: Automatic half-precision inference for 2x speed on T4/A100 GPUs.
- **Self-Contained Deployment**: Generates a robust FastAPI server script dynamically.
- **Blueprint Visualization**: Professional-grade plotting with synchronized legends.

## 🎯 Prerequisites
1. **GPU Runtime**: T4 or better (Runtime → Change runtime type → GPU)
2. **Model**: A trained YOLOv11-OBB model (`.pt` file).
3. **Ngrok Token**: For exposing the API publicly (Optional).

In [ ]:
# Mount Google Drive for model access
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
    print("✅ Drive mounted at: /content/drive/MyDrive")
else:
    print("✅ Drive already mounted")

In [ ]:
print("="*70)
print("🔧 Environment Setup & Optimization")
print("="*70)

# Install optimized dependencies
print("\n📦 Installing libraries...")
!pip install -q ultralytics>=8.3.0 fastapi>=0.115.0 uvicorn[standard]>=0.34.0
!pip install -q python-multipart pyngrok>=7.0.0 python-dotenv opencv-python-headless

import sys
import torch
import cv2
import numpy as np
from ultralytics import YOLO

# System Validation
print(f"\n🐍 Python: {sys.version.split()[0]}")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"👁️ OpenCV: {cv2.__version__}")

DEVICE = 'cpu'
if torch.cuda.is_available():
    DEVICE = 'cuda'
    gpu_name = torch.cuda.get_device_name(0)
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\n✅ GPU Detected: {gpu_name} ({total_mem:.2f} GB)")

    # Enable cuDNN benchmark for optimized performance on fixed size inputs
    torch.backends.cudnn.benchmark = True
    print("   🚀 cuDNN Benchmark Enabled")
else:
    print("\n⚠️  WARNING: No GPU detected. Inference will be slow.")

print("\n✅ Environment Ready!")

In [ ]:
import os
import torch

print("⚙️  Pipeline Configuration")
print("="*70)

# --- USER CONFIGURATION START ---
# Path to your YOLOv11-OBB model
MODEL_PATH = "/content/drive/Shareddrives/HVAC/DECEMBER 24 OUTPUT WEIGHTS {dataset2}/hvac_obb_l_20251224_214011/weights/best.pt"

# Inference Settings
CONF_THRESHOLD = 0.50      # Confidence threshold (0.0 - 1.0)
IOU_THRESHOLD = 0.45       # NMS IoU threshold
IMG_SIZE = 1024            # Inference image size (pixels)
HALF_PRECISION = torch.cuda.is_available() # Use FP16 if GPU is available

# Server Settings
PORT = 8000
NGROK_AUTHTOKEN = "YOUR_NGROK_TOKEN_HERE" # <--- PASTE TOKEN HERE
# --- USER CONFIGURATION END ---

# Validation logic
if not os.path.exists(MODEL_PATH):
    print(f"❌ ERROR: Model not found at {MODEL_PATH}")
    print("   Please update MODEL_PATH to point to your .pt file.")
else:
    print(f"✅ Model Path: {MODEL_PATH}")

print(f"🎯 Configuration: Conf={CONF_THRESHOLD}, IoU={IOU_THRESHOLD}, Size={IMG_SIZE}, FP16={HALF_PRECISION}")

In [ ]:
import time

print("🧠 Model Loading & Initialization")
print("="*70)

try:
    start_t = time.time()
    model = YOLO(MODEL_PATH)

    # Check if model supports OBB
    task = model.task
    print(f"   Detected Task: {task.upper()}")
    if task != 'obb':
        print("⚠️  WARNING: This does not appear to be an OBB model. Results may vary.")

    # Move to GPU
    model.to(DEVICE)
    print(f"✅ Model loaded on {DEVICE} in {time.time()-start_t:.2f}s")

    # Print Classes
    print(f"\n📚 Classes ({len(model.names)}):")
    for i, name in model.names.items():
        if i < 5: print(f"   - {name}")
    if len(model.names) > 5: print("   ... (and others)")

    # Warmup
    print("\n🔥 Warming up GPU...")
    dummy_input = np.zeros((640, 640, 3), dtype=np.uint8)
    model.predict(dummy_input, verbose=False, half=HALF_PRECISION)
    print("✅ Warmup complete")

except Exception as e:
    print(f"❌ Fatal Error loading model: {e}")
    raise

In [ ]:
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

print("🖼️  Interactive Inference & OBB Visualization")
print("="*70)

def run_inference_pipeline():
    # 1. Upload Image
    print("\n📤 Please upload an HVAC diagram image...")
    uploaded = files.upload()
    if not uploaded: return

    filename = list(uploaded.keys())[0]

    # 2. Preprocessing
    original_pil = Image.open(filename).convert('RGB')
    img_np = np.array(original_pil)

    # 3. Inference (Optimized)
    print(f"\n🔄 Analyzing {filename}...")
    start_inf = time.time()
    results = model.predict(
        img_np,
        conf=CONF_THRESHOLD,
        iou=IOU_THRESHOLD,
        imgsz=IMG_SIZE,
        half=HALF_PRECISION,
        verbose=False
    )
    end_inf = time.time()
    result = results[0]

    # 4. Process OBB Results
    # Create a copy for drawing
    draw_img = img_np.copy()

    # Generate distinct colors for classes
    np.random.seed(42)
    color_map = {id: tuple(np.random.randint(0, 255, 3).tolist()) for id in result.names}

    detections_found = 0
    class_counts = {}

    # Check for OBB data first, fallback to Boxes
    if hasattr(result, 'obb') and result.obb is not None:
        boxes_data = result.obb
        is_obb = True
    else:
        boxes_data = result.boxes
        is_obb = False
        print("⚠️  No OBB data found, falling back to standard boxes.")

    if boxes_data is not None:
        for box in boxes_data:
            cls_id = int(box.cls[0].item())
            cls_name = result.names[cls_id]
            conf = box.conf[0].item()
            color = color_map[cls_id]

            # Update stats
            detections_found += 1
            class_counts[cls_name] = class_counts.get(cls_name, 0) + 1

            # Draw Logic
            if is_obb:
                # OBB returns xyxyxyxy (4 points)
                # tensor shape: [4, 2]
                pts = box.xyxyxyxy[0].cpu().numpy().astype(np.int32)
                pts = pts.reshape((-1, 1, 2))
                cv2.polylines(draw_img, [pts], isClosed=True, color=color, thickness=3)
            else:
                # Standard Box
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                cv2.rectangle(draw_img, (x1, y1), (x2, y2), color, 3)

    # 5. Visualization with Matplotlib
    fig = plt.figure(figsize=(20, 10))
    gs = fig.add_gridspec(1, 2, width_ratios=[3, 1])
    ax_img = fig.add_subplot(gs[0])
    ax_legend = fig.add_subplot(gs[1])

    # Image Plot
    ax_img.imshow(draw_img)
    ax_img.axis('off')
    ax_img.set_title(f"Inference Result ({detections_found} detections) | {(end_inf-start_inf)*1000:.1f}ms", fontsize=14)

    # Legend Plot
    legend_elements = []
    sorted_counts = dict(sorted(class_counts.items(), key=lambda item: item[1], reverse=True))

    for name, count in sorted_counts.items():
        # Find ID for name
        cid = [k for k, v in result.names.items() if v == name][0]
        c_norm = [c/255 for c in color_map[cid]] # Normalize for matplotlib
        legend_elements.append(Patch(facecolor=c_norm, edgecolor='black', label=f"{name}: {count}"))

    ax_legend.axis('off')
    if legend_elements:
        ax_legend.legend(handles=legend_elements, loc='center', title="Component Counts", fontsize=12, title_fontsize=14)
    else:
        ax_legend.text(0.5, 0.5, "No Detections", ha='center', fontsize=12)

    plt.tight_layout()
    plt.savefig('obb_result.png', dpi=150)
    plt.show()

    print(f"✅ Results saved to obb_result.png")

# Run the pipeline
run_inference_pipeline()

In [ ]:
print("📝 Generating Production Server Code (app.py)...")
print("="*70)

# We use double braces {{ }} for python dictionaries inside the f-string
# to prevent SyntaxErrors.

server_code = f"""
import uvicorn
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import numpy as np
import cv2
import torch
from ultralytics import YOLO
import logging
import sys

# --- CONFIGURATION ---
MODEL_PATH = r'{MODEL_PATH}'
CONF_THRES = {CONF_THRESHOLD}
IOU_THRES = {IOU_THRESHOLD}
IMG_SIZE = {IMG_SIZE}
HALF = {HALF_PRECISION}

# --- LOGGING ---
# Stream logs to stdout so Colab can see them
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger("HVAC-Service")

app = FastAPI(title="HVAC YOLOv11-OBB Inference API")

model = None

@app.on_event("startup")
async def load_model():
    global model
    logger.info(f"Loading model from {{MODEL_PATH}}...")
    try:
        model = YOLO(MODEL_PATH)
        if torch.cuda.is_available():
            model.to('cuda')
            logger.info("Model loaded on GPU")
        else:
            logger.info("Model loaded on CPU")

        # Warmup
        model.predict(np.zeros((640,640,3), dtype=np.uint8), verbose=False, half=HALF)
        logger.info("Model Warmup Complete")
    except Exception as e:
        logger.error(f"Failed to load model: {{e}}")
        raise RuntimeError("Model loading failed")

@app.get("/")
def root():
    return {{"message": "HVAC Inference Server Online. Use /analyze endpoint."}}

@app.get("/health")
def health_check():
    if model is None:
        raise HTTPException(status_code=503, detail="Model initializing")
    return {{"status": "healthy", "device": str(model.device)}}

@app.post("/analyze")
async def analyze_image(file: UploadFile = File(...)):
    if not model:
        raise HTTPException(status_code=503, detail="Model not loaded")

    try:
        contents = await file.read()
        nparr = np.frombuffer(contents, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if img is None:
            raise HTTPException(status_code=400, detail="Invalid image format")

        results = model.predict(
            img,
            conf=CONF_THRES,
            iou=IOU_THRES,
            imgsz=IMG_SIZE,
            half=HALF
        )

        result = results[0]
        detections = []

        # Handle OBB vs Standard
        if hasattr(result, 'obb') and result.obb is not None:
            for box in result.obb:
                # xywhr: x_center, y_center, width, height, rotation
                r_box = box.xywhr[0].cpu().numpy().tolist()
                cls_id = int(box.cls[0].item())
                conf = float(box.conf[0].item())

                detections.append({{
                    "class": result.names[cls_id],
                    "confidence": conf,
                    "type": "OBB",
                    "bbox": {{
                        "x_center": r_box[0],
                        "y_center": r_box[1],
                        "width": r_box[2],
                        "height": r_box[3],
                        "rotation": r_box[4]
                    }}
                }})
        else:
            # Fallback for standard rect models
            for box in result.boxes:
                xyxy = box.xyxy[0].cpu().numpy().tolist()
                cls_id = int(box.cls[0].item())
                conf = float(box.conf[0].item())
                detections.append({{
                    "class": result.names[cls_id],
                    "confidence": conf,
                    "type": "RECT",
                    "bbox": xyxy
                }})

        return JSONResponse(content={{
            "count": len(detections),
            "detections": detections
        }})

    except Exception as e:
        logger.error(f"Inference error: {{e}}")
        raise HTTPException(status_code=500, detail=str(e))
"""

with open("app.py", "w") as f:
    f.write(server_code)

print("✅ Generated app.py successfully")

In [ ]:
import subprocess
import time
import requests
import sys
from pyngrok import ngrok

print("🚀 Launching & Validating Server")
print("="*70)

# 1. Cleanup existing tunnels
ngrok.kill()

# 2. Start Uvicorn in Background (using Popen to keep control)
print("⏳ Starting Uvicorn process...")
# We use PIPE to capture stdout so we can print it later
process = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "app:app", "--host", "0.0.0.0", "--port", str(PORT)],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    encoding='utf-8',
    bufsize=1
)

# 3. Health Check Loop
print("🏥 Checking server health (timeout: 60s)...\n")
server_ready = False
health_url = f"http://localhost:{PORT}/health"

start_time = time.time()
while time.time() - start_time < 60:
    try:
        # Try to connect
        response = requests.get(health_url, timeout=1)
        if response.status_code == 200:
            data = response.json()
            print(f"\n✅ Server is HEALTHY!")
            print(f"   Status: {data['status']}")
            print(f"   Device: {data['device']}")
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        # Server not up yet, print dot and wait
        print(".", end="", flush=True)
        time.sleep(2)
    except Exception as e:
        print(f"\n⚠️ Unexpected error during check: {e}")

if not server_ready:
    print("\n❌ Server failed to start within timeout.")
    print("--- Server Logs ---")
    print(process.stdout.read()) # Dump logs to see why it failed
    process.terminate()
    raise RuntimeError("Server startup failed")

# 4. Setup Ngrok (Only if healthy)
print("\n🌐 Initializing Public Tunnel...")
if NGROK_AUTHTOKEN and NGROK_AUTHTOKEN != "YOUR_NGROK_TOKEN_HERE":
    try:
        ngrok.set_auth_token(NGROK_AUTHTOKEN)
        public_url = ngrok.connect(PORT).public_url
        print(f"\n🎉 API IS LIVE at: {public_url}")
        print(f"   📄 Docs: {public_url}/docs")
        print(f"   🔍 Test: {public_url}/analyze")
    except Exception as e:
        print(f"⚠️ Ngrok Error: {e}")
else:
    print("⚠️ No Ngrok token. Server local at http://localhost:8000")

print("\n📜 Streaming Logs (Press STOP to exit)...")
print("-"*70)

# 5. Log Streaming Loop
try:
    while True:
        # Read line from process
        line = process.stdout.readline()
        if line:
            print(line.strip())
        # Check if process died
        if process.poll() is not None:
            print("❌ Server process terminated unexpectedly.")
            break
except KeyboardInterrupt:
    print("\n🛑 Server stopped by user.")
    process.terminate()
    ngrok.kill()